In [ ]:
import numpy as np
import cv2
from copy import deepcopy
from PIL import Image
import pytesseract
import sys
import urllib
#pytesseract.pytesseract.tesseract_cmd = "/root/.cache/pip/wheels/df/0f/58/4577e860a694ad2a77cddf2f17bb77e9d21ad7f0758a4bc6f9"
#pytesseract.tesseract_cmd = r'/usr/local/bin/tesseract'
#pytesseract.tesseract_cmd = r'/usr/local/bin/pytesseract'
#pytesseract.tesseract_cmd = r'/usr/local/bin/tesseract-ocr'
#pytesseract.tesseract_cmd = r'/usr/local/bin/libtesseract-dev'



def process(img):
  #cv2.imshow("Input",img)
  imgBlurred = cv2.GaussianBlur(img, (5,5), 0)
  gray = cv2.cvtColor(imgBlurred, cv2.COLOR_BGR2GRAY)

  sobelx = cv2.Sobel(gray,cv2.CV_8U,1,0,ksize=3)
  #cv2.imshow("Sobel",sobelx)
  #cv2.waitKey(0)
  ret2,threshold_img = cv2.threshold(sobelx,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
  #cv2.imshow("Threshold",threshold_img)
  #cv2.waitKey(0)
  return threshold_img

def choose_largest(plate):
  print ("CLEANING PLATE. . .")
  gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
  #kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
  #thresh= cv2.dilate(gray, kernel, iterations=1)

  _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
  im1,contours,hierarchy = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

  if contours:
    areas = [cv2.contourArea(c) for c in contours]
    max_index = np.argmax(areas)

    max_cnt = contours[max_index]
    max_cntArea = areas[max_index]
    x,y,w,h = cv2.boundingRect(max_cnt)
    if not checkRatio(max_cntArea,w,h):
      return plate,None

    cleaned_final = thresh[y:y+h, x:x+w]
    #cv2.imshow("Function Test",cleaned_final)
    return cleaned_final,[x,y,w,h]

  else:
    return plate,None


def get_edges(threshold_img):
  element = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(17, 3))
  morph_img_threshold = threshold_img.copy()
  cv2.morphologyEx(src=threshold_img, op=cv2.MORPH_CLOSE, kernel=element, dst=morph_img_threshold)
  #cv2.imshow("Morphed",morph_img_threshold)
  cv2.waitKey(0)

  im2,contours, hierarchy= cv2.findContours(morph_img_threshold,mode=cv2.RETR_EXTERNAL,method=cv2.CHAIN_APPROX_NONE)
  return contours


def checkRatio(area, width, height):
  ratio = float(width) / float(height)
  if ratio < 1:
    ratio = 1 / ratio

  aspect = 4.7272
  min = 15*aspect*15  # minimum area
  max = 125*aspect*125  # maximum area

  rmin = 3
  rmax = 6

  if (area < min or area > max) or (ratio < rmin or ratio > rmax):
    return False
  return True

def checkBlankness(plate):
  avg = np.mean(plate)
  if(avg>=115):
    return True
  else:
    return False

def validation_check(rect):
  (x, y), (width, height), rect_angle = rect
  if(width>height):
    angle = -rect_angle
  else:
    angle = 90 + rect_angle

  if angle>15:
    return False

  if height == 0 or width == 0:
    return False

  area = height*width
  if not checkRatio(area,width,height):
    return False
  else:
    return True




def choose_plate(img,contours):
  #count=0
  for i,cnt in enumerate(contours):
    min_rect = cv2.minAreaRect(cnt)
    if validation_check(min_rect):

      x,y,w,h = cv2.boundingRect(cnt)
      plate_img = img[y:y+h,x:x+w]

      if(checkBlankness(plate_img)):
        #count+=1
        clean_plate, rect = choose_largest(plate_img)
        if rect:
          x1,y1,w1,h1 = rect
          x,y,w,h = x+x1,y+y1,w1,h1
          #cv2.imshow("Cleaned Plate",clean_plate)
          cv2.waitKey(0)
          img = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
          area=(x,y,(x+w),(y+h))
          get_chars(img,area)
          break

def get_chars(img,area):
  cv2.imwrite("drive/My Drive/Colab/testp.jpg",img)
  img1=Image.open("drive/My Drive/Colab/testp.jpg")
  img1=img1.crop(area)
  img1.save("drive/My Drive/Colab/testf.jpg")
#   img=cv2.imread("drive/My Drive/Colab/17.jpg")
#   gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#   gray = cv2.medianBlur(gray, 3)
#   cv2.imwrite("drive/My Drive/Colab/testf.jpg", gray)
  text = pytesseract.image_to_string(Image.open("drive/My Drive/Colab/testf.jpg"), lang='eng')
  if(len(text)<5):
    gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    cv2.imwrite("drive/My Drive/Colab/testf.jpg", gray)
    text = pytesseract.image_to_string(Image.open("drive/My Drive/Colab/testf.jpg"), lang='eng')
#   baseURL='https://api.thingspeak.com/update?api_key=UP2MYEJSHUGSTV7D&field1='
#   f = urllib.request.urlopen(baseURL + str(text))
#   f.read()
#   f.close()
  print("Detected Text : ",text)

          #cv2.waitKey(0)

	#print "No. of final cont : " , count



if __name__ == '__main__':
  print ("DETECTING PLATE . . .")

  #img = cv2.imread("testData/Final.JPG")
  img = cv2.imread("drive/My Drive/Colab/test1.jpg")
  threshold_img = process(img)
  contours= get_edges(threshold_img)

	#if len(contours)!=0:
		#print len(contours) #Test
		# cv2.drawContours(img, contours, -1, (0,255,0), 1)
		# cv2.imshow("Contours",img)
		# cv2.waitKey(0)


  choose_plate(img,contours)

DETECTING PLATE . . .
CLEANING PLATE. . .
Detected Text :  TH07BU54:27


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!pip install pyrebase

     |████████████████████████████████| 460kB 7.1MB/s 
     |████████████████████████████████| 81kB 27.8MB/s 
     |████████████████████████████████| 6.5MB 19.5MB/s 
     |████████████████████████████████| 522kB 42.2MB/s 
     |████████████████████████████████| 61kB 25.1MB/s 
  Created wheel for gcloud: filename=gcloud-0.17.0-cp36-none-any.whl size=638015 sha256=3b7c1cb7e1f13facf1af3c567f7474fb493ad72da9a579f34d093707b36534d4
  Stored in directory: /root/.cache/pip/wheels/b9/04/ff/66c87c7e2419fd477f52015468ec5301d9480b6f3896babda4
  Created wheel for oauth2client: filename=oauth2client-3.0.0-cp36-none-any.whl size=106382 sha256=495260da98043f078002a946f5e432745c5cf91739e1dba3311d2fca4f24f385
  Stored in directory: /root/.cache/pip/wheels/48/f7/87/b932f09c6335dbcf45d916937105a372ab14f353a9ca431d7d
  Created wheel for pycryptodome: filename=pycryptodome-3.4.3-cp36-cp36m-linux_x86_64.whl size=6813150 sha256=78a02dc450562eaef34dc337976e6c3dd3e452e609cbced94aea5e5f2b383992
  Stored in direc

In [ ]:
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 8 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 3s (1,400 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../tesseract

In [ ]:
!pip install pytesseract

  Created wheel for pytesseract: filename=pytesseract-0.3.0-py2.py3-none-any.whl size=20970 sha256=eb136826ead527bb72c8a5fc7ad9d80d3dacc6b10559ad0bed8c6ec4d019f2d3
  Stored in directory: /root/.cache/pip/wheels/78/c9/ac/4cb76bd547f9970070522439e5203ba7926c5c5c4f131583ea
Successfully built pytesseract
